In [69]:
import pandas as pd
import numpy as np

In [70]:
df = pd.read_excel("[MASA Hackathon 2022] Travel Insurance Data Set .xlsx")
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,NaN,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,NaN,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,NaN,41


In [71]:
df.describe(include="all")

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
count,63326,63326,63326,63326,63326,63326.000000,63326,63326.000000,63326.000000,18219,63326.000000
unique,16,2,2,26,2,NaN,149,NaN,NaN,2,NaN
top,EPX,Travel Agency,Online,Cancellation Plan,No,NaN,SINGAPORE,NaN,NaN,M,NaN
freq,35119,45869,62219,18630,62399,NaN,13255,NaN,NaN,9347,NaN
mean,NaN,NaN,NaN,NaN,NaN,49.317074,NaN,40.702018,9.809992,NaN,39.969981
std,NaN,NaN,NaN,NaN,NaN,101.791566,NaN,48.845637,19.804388,NaN,14.017010
min,NaN,NaN,NaN,NaN,NaN,-2.000000,NaN,-389.000000,0.000000,NaN,0.000000
25%,NaN,NaN,NaN,NaN,NaN,9.000000,NaN,18.000000,0.000000,NaN,35.000000
50%,NaN,NaN,NaN,NaN,NaN,22.000000,NaN,26.530000,0.000000,NaN,36.000000
75%,NaN,NaN,NaN,NaN,NaN,53.000000,NaN,48.000000,11.550000,NaN,43.000000


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63326 entries, 0 to 63325
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agency                63326 non-null  object 
 1   Agency Type           63326 non-null  object 
 2   Distribution Channel  63326 non-null  object 
 3   Product Name          63326 non-null  object 
 4   Claim                 63326 non-null  object 
 5   Duration              63326 non-null  int64  
 6   Destination           63326 non-null  object 
 7   Net Sales             63326 non-null  float64
 8   Commision (in value)  63326 non-null  float64
 9   Gender                18219 non-null  object 
 10  Age                   63326 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 5.3+ MB


In [73]:
df.shape

(63326, 11)

Observations:
1. [Gender] has a lot of missing data.
2. The minimum value for [Duration] is -2, which doesn't make sense.
3. The minimum value for [Net Sales] is negative, which doesn't make sense too. 
4. The maximum [Age] is 118, which is not very common.
5. The data has 63326 rows and 11 columns

In [74]:
#function to find number of outliers
def outliers(df):
    q1=df.quantile(0.25)
    q3=df.quantile(0.75)
    IQR=q3-q1
    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]
    return len(outliers)

### [Gender]

Since genders has so many missing values and we couldn't find a way to fill in those null values, we will just drop that column.

In [75]:
df.drop(["Gender"], axis = 1, inplace = True)
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,41


### [Age]

In [76]:
print("Number of unique values :", len(df["Age"].unique()))
print(df["Age"].value_counts(bins=10))

Number of unique values : 89
(35.4, 47.2]      33341
(23.6, 35.4]      15521
(47.2, 59.0]       8459
(59.0, 70.8]       2546
(11.8, 23.6]       1780
(106.2, 118.0]      984
(70.8, 82.6]        588
(82.6, 94.4]         63
(-0.119, 11.8]       44
(94.4, 106.2]         0
Name: Age, dtype: int64


There is no one at the age from 94.4 to 106.2, but there is 984 people at the age of 106.2 to 118. We are going to remove those 984 outliers in this case.

In [77]:
df = df[df["Age"] < 100]
print(df["Age"].value_counts(bins=10))

(35.2, 44.0]     30985
(26.4, 35.2]     13120
(44.0, 52.8]      7189
(52.8, 61.6]      4333
(17.6, 26.4]      4096
(61.6, 70.4]      1839
(70.4, 79.2]       527
(79.2, 88.0]       124
(8.8, 17.6]        102
(-0.089, 8.8]       27
Name: Age, dtype: int64


### [Duration]

In [78]:
df.loc[df['Duration'] <= 0]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age
911,RAB,Airlines,Online,Value Plan,No,0,BRUNEI DARUSSALAM,15.00,6.00,24
2515,JZI,Airlines,Online,Basic Plan,No,0,MYANMAR,18.00,6.30,26
2868,JZI,Airlines,Online,Basic Plan,No,0,INDONESIA,18.00,6.30,42
3282,RAB,Airlines,Online,Value Plan,No,0,BRUNEI DARUSSALAM,15.00,6.00,38
3577,SSI,Airlines,Online,Ticket Protector,No,0,SINGAPORE,14.40,4.04,48
3903,JZI,Airlines,Online,Basic Plan,No,0,MYANMAR,18.00,6.30,27
4281,JZI,Airlines,Online,Basic Plan,No,0,MYANMAR,18.00,6.30,24
6171,JZI,Airlines,Online,Basic Plan,No,0,THAILAND,18.00,6.30,24
8490,SSI,Airlines,Online,Ticket Protector,No,0,SINGAPORE,9.77,2.74,48
9685,C2B,Airlines,Online,Silver Plan,No,0,SINGAPORE,0.00,5.63,51


Some values of [Duration] are zero which does not make sense.

In [79]:
df_duration = df["Duration"]
df_duration.head()

0    186
1    186
2     65
3     60
4     79
Name: Duration, dtype: int64

In [80]:
df_duration.describe()

count    62342.000000
mean        49.639136
std        102.465902
min          0.000000
25%         10.000000
50%         23.000000
75%         53.000000
max       4881.000000
Name: Duration, dtype: float64

In [81]:
print("outliers of [Duration]: ", outliers(df_duration))

outliers of [Duration]:  5659


From the results above, there are a total of 5659 outliers in the column [Duration] out of 62342. The SD is very high and the data is skewed heavily towards the right. Therefore, we are replacing the values of zero in the [Duration] column with the median.

In [82]:
df["Duration"] = df["Duration"].replace(0, df["Duration"].quantile(0.5))

In [83]:
df.describe()

,Duration,Net Sales,Commision (in value),Age
count,62342.000000,62342.000000,62342.000000,62342.000000
mean,49.652786,40.560025,9.661998,38.738363
std,102.460820,48.883541,19.804959,10.097333
min,1.000000,-389.000000,0.000000,0.000000
25%,10.000000,18.000000,0.000000,35.000000
50%,23.000000,26.000000,0.000000,36.000000
75%,53.000000,48.000000,10.500000,43.000000
max,4881.000000,682.000000,262.760000,88.000000


By looking at the first column, the minimum value of [Duration] is now 1.

### [Net Sales] and [Comission]

Finding the relationship between these two features

In [84]:
df.loc[df["Commision (in value)"]<0]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age


It seems that there are no negative values in [Commision]

In [85]:
df.loc[df['Net Sales'] < 0]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,-29.0,9.57,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,-29.0,9.57,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65.0,AUSTRALIA,-49.5,29.70,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60.0,AUSTRALIA,-39.6,23.76,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79.0,ITALY,-19.8,11.88,41
...,...,...,...,...,...,...,...,...,...,...
57991,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,6.0,AUSTRALIA,-9.9,5.94,31
57992,RAB,Airlines,Online,Value Plan,No,9.0,BRUNEI DARUSSALAM,-15.0,6.00,46
57993,C2B,Airlines,Online,Annual Silver Plan,No,386.0,SINGAPORE,-161.0,40.25,28
57994,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,28.0,MALAYSIA,-29.7,17.82,31


Several values of [Net Sales] are smaller than zero, which does not make sense.

In [86]:
#Calculating the percentage of comission by using the absolute value of [Net Sales]
df["Commision Percentage"] = (df["Commision (in value)"]/(df["Commision (in value)"] + abs (df["Net Sales"])))
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age,Commision Percentage
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,-29.0,9.57,81,0.24812
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,-29.0,9.57,71,0.24812
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65.0,AUSTRALIA,-49.5,29.70,32,0.37500
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60.0,AUSTRALIA,-39.6,23.76,32,0.37500
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79.0,ITALY,-19.8,11.88,41,0.37500


In [87]:
df.groupby("Agency")["Commision Percentage"].std()

Agency
ADM    2.676453e-01
ART    1.253861e-01
C2B    1.035846e-01
CBH    1.639111e-01
CCR    1.066732e-01
CSR    5.214365e-18
CWT    2.000606e-01
EPX    0.000000e+00
JZI    1.011546e-01
KML    3.659973e-02
LWC    1.069366e-01
RAB    7.466864e-02
SSI    1.714436e-03
TST    7.858775e-02
TTW    0.000000e+00
Name: Commision Percentage, dtype: float64

In [88]:
#df.loc[df['Agency'] == "ADM"]
#df.loc[df['Agency'] == "C2B"]
df_agency = df.loc[df['Agency'] == "CWT"]
df_agency.head(30)

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age,Commision Percentage
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65.0,AUSTRALIA,-49.5,29.70,32,0.375
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60.0,AUSTRALIA,-39.6,23.76,32,0.375
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79.0,ITALY,-19.8,11.88,41,0.375
6,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,47.0,UNITED STATES,-39.6,23.76,32,0.375
7,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,63.0,AUSTRALIA,-108.9,65.34,29,0.375
8,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,57.0,THAILAND,-19.8,11.88,44,0.375
9,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,186.0,AUSTRALIA,-99.0,59.40,37,0.375
13,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,5.0,MALAYSIA,-39.6,23.76,64,0.375
15,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,6.0,DENMARK,-19.8,11.88,53,0.375
16,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,48.0,DENMARK,-79.2,47.52,43,0.375


The above shows that the [Commision Percentage] are the same within the same [Agency] if we use the absolute value of [Net Sales] to calculate the result. Moreover, from index 13 and 128, we can see that the absolute value of both [Net Sales] are the same. This induces that the negative value could be a typing error. Therefore, we are going to replace all the [Net Sales] with its absolute value.

In [89]:
df["Net Sales"] = df["Net Sales"].apply(lambda x : abs(x))
df_agency = df.loc[df['Agency'] == "CWT"]
df_agency.head(30)

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age,Commision Percentage
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65.0,AUSTRALIA,49.5,29.70,32,0.375
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60.0,AUSTRALIA,39.6,23.76,32,0.375
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79.0,ITALY,19.8,11.88,41,0.375
6,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,47.0,UNITED STATES,39.6,23.76,32,0.375
7,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,63.0,AUSTRALIA,108.9,65.34,29,0.375
8,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,57.0,THAILAND,19.8,11.88,44,0.375
9,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,186.0,AUSTRALIA,99.0,59.40,37,0.375
13,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,5.0,MALAYSIA,39.6,23.76,64,0.375
15,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,6.0,DENMARK,19.8,11.88,53,0.375
16,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,48.0,DENMARK,79.2,47.52,43,0.375


We also spotted a few 0 values in [Net Sales] with their commision not equal to 0. Let's take a look at these special cases.

In [90]:
df.loc[df["Net Sales"] == 0]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age,Commision Percentage
31,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,54.0,AUSTRALIA,0.0,41.58,31,1.0
80,EPX,Travel Agency,Online,Cancellation Plan,No,38.0,"KOREA, REPUBLIC OF",0.0,0.00,36,NaN
87,EPX,Travel Agency,Online,Cancellation Plan,No,8.0,VIET NAM,0.0,0.00,36,NaN
129,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,5.0,MALAYSIA,0.0,29.70,53,1.0
144,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,201.0,AUSTRALIA,0.0,77.22,23,1.0
...,...,...,...,...,...,...,...,...,...,...,...
62806,C2B,Airlines,Online,Bronze Plan,No,12.0,SINGAPORE,0.0,6.63,24,1.0
62833,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,165.0,AUSTRALIA,0.0,29.70,26,1.0
62993,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,5.0,MALAYSIA,0.0,17.82,53,1.0
62996,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,3.0,MALAYSIA,0.0,17.82,49,1.0


There are 1866 rows that has 0 [Net Sales]. Let's replace them by reverse calculation, using [Net Sales] = [Comission] / [Comission Percentage] - [Commision]

In [91]:
#Finding the mode [Commision Percentage] of every [Agency]
percentage_commision = df.groupby("Agency")["Commision Percentage"].agg(pd.Series.mode)
percentage_commision

Agency
ADM                                      0.275362
ART                                      0.259259
C2B                                           0.2
CBH                                       0.24812
CCR                                       0.24812
CSR                                       0.24812
CWT                                         0.375
EPX                                           0.0
JZI                                      0.259259
KML                                      0.275362
LWC                                      0.393939
RAB                                      0.285714
SSI    [0.21965317919075145, 0.21969696969696967]
TST                                      0.259259
TTW                                           0.0
Name: Commision Percentage, dtype: object

In [92]:
percentage_commision.loc["SSI"] = percentage_commision.loc["SSI"][0]

In [93]:
percentage_commision = pd.core.frame.DataFrame(percentage_commision)
percentage_commision = percentage_commision.reset_index()
percentage_commision.head()

,Agency,Commision Percentage
0,ADM,0.275362
1,ART,0.259259
2,C2B,0.2
3,CBH,0.24812
4,CCR,0.24812


In [94]:
#Dropping the initial [Commision Percentage] and replacing it with the mode of the initial [Commision Percentage] 
#of different Agency
df.drop(["Commision Percentage"], axis=1, inplace = True)
df = df.merge(percentage_commision[["Agency", "Commision Percentage"]], 'left')
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age,Commision Percentage
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,29.0,9.57,81,0.24812
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,29.0,9.57,71,0.24812
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65.0,AUSTRALIA,49.5,29.70,32,0.375
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60.0,AUSTRALIA,39.6,23.76,32,0.375
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79.0,ITALY,19.8,11.88,41,0.375


In [95]:
for i in range(df.shape[0]):
    if (df.loc[i, "Agency"] != "EPX") and (df.loc[i, "Agency"] != "TTW"):
        df.loc[i, "Net Sales"] = df.loc[i, "Commision (in value)"] / df.loc[i, "Commision Percentage"] - df.loc[i, "Commision (in value)"]
    else:
        continue
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age,Commision Percentage
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,29.0,9.57,81,0.24812
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,29.0,9.57,71,0.24812
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65.0,AUSTRALIA,49.5,29.70,32,0.375
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60.0,AUSTRALIA,39.6,23.76,32,0.375
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79.0,ITALY,19.8,11.88,41,0.375


In [96]:
#Changing the columns' position
df = df[["Agency", "Agency Type", "Distribution Channel", "Product Name", "Claim", "Duration", "Destination", 
         "Age", "Net Sales", "Commision (in value)", "Commision Percentage"]]
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Age,Net Sales,Commision (in value),Commision Percentage
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,81,29.0,9.57,0.24812
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186.0,MALAYSIA,71,29.0,9.57,0.24812
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65.0,AUSTRALIA,32,49.5,29.70,0.375
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60.0,AUSTRALIA,32,39.6,23.76,0.375
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79.0,ITALY,41,19.8,11.88,0.375


In [97]:
df.shape

(62342, 11)

After data cleaning, we are left with 62342 rows of data

In [103]:
# exporting the cleaned data into an excel file
df.to_excel("[MASA Hackathon 2022] Travel Insurance Data Set (Cleaned).xlsx", index = False);